In [1]:
import pandas as pd

In [14]:
clin = pd.read_csv('../data/data/TCGA_Metabric_clinical.csv', index_col=0)

In [36]:
for target in list(clin):
    kept = clin[~clin[target].isna()][[target]]
    kept.to_csv('../data/TCGA_Metabric_'+target.replace(' ', '_')+'.csv')

# Define custom models

In [91]:
class MyMLPClassifier(MLPClassifier):
    def __init__(self, hidden_layer_width=2, hidden_layer_depth=2, **kwargs):
        hidden_layer_sizes = tuple([ int(hidden_layer_width)] * int(hidden_layer_depth))
        super().__init__(hidden_layer_sizes=hidden_layer_sizes, **kwargs)


# Import MongoDB credentials

In [7]:
import csv
with open('credentials.csv') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        mongo_user = row['username']
        mongo_pwd = row['pwd']
        mongo_host = row['host']        

# Usage

In [10]:
import prior_pipeline as pp

from hyperopt import hp
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.gaussian_process.kernels import RBF


mongo = {
            'url': 'mongo://{}:{}@{}:27017/hyperopt/jobs'.format(mongo_user, mongo_pwd, mongo_host),
            'exp': 'test_run'
        }

mongo = {
            'url': 'mongo://{}:27017/hyperopt/jobs'.format(mongo_host),
            'exp': 'test_run'
        }

data_paths = ['./data/TCGA_Metabric_RNA_TSNE_{}.csv'.format(i) for i in [3, 5, 10, 15, 20, 40, 50]]
targets_path =  './data/TCGA_Metabric_2_years.csv'

space = (hp.choice('model',
        [
            {
                'name': SVC,
                'params':{
                    'probability':True
                }
            },
#             {
#                 'name': GaussianProcessClassifier,
#                 'params': {
#                     'kernel':  RBF(1),
#                 }
#             },
            {
                'name': GaussianNB
            },
#             {
#             'name': MyMLPClassifier,
#             'params': {
#                 'activation': hp.choice('activation', ['tanh', 'relu']),
#                 'alpha': hp.choice('alpha', [5e-2]),
#                 'solver' : hp.choice('solver', ['adam', 'lbfgs', 'sgd']),
#                 'learning_rate': hp.choice('learning_rate', ['constant', 'adaptive']),
#                 'hidden_layer_width': hp.quniform('hidden_layer_width', 2, 300, 1),
#                 'hidden_layer_depth': hp.quniform('hidden_layer_depth', 2, 4, 1)
#             }
#         }
        ]),
        hp.choice('data_path', data_paths),
        targets_path)


pp.classifier.fit(space=space, max_evals=10, mongo=mongo)

ValueError: ('unrecognized protocol for MongoJobs', 'mongodb')